<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/RAG_RespondendoPergunta_Llama2_13B_4bit_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Respondendo Perguntas sobre Textos Longos usando Llama v2.0, Longchain, Chroma e Transformers by HuggingFace

**Toda a execução ocorre no Google Colaboratory.**

Pré-requisitos:
- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Referências**

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

**Lista dos modelos:**

https://huggingface.co/models


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Imprime linhas menores.

In [ ]:
def print_linhas_menores(texto, tamanho=120):
  for i in range(0, len(texto), tamanho):
    print(texto[i:i+tamanho])

# 1 - Instalação das bibliotecas

Biblioteca para manipular pdf

https://pypi.org/project/pypdf/

In [ ]:
!pip install pypdf==3.16.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 5.7 MB/s eta 0:00:00


Bibliota de dependência para manipular os embeddings pelo Langchain.

https://pypi.org/project/sentence-transformers/

In [ ]:
!pip install sentence_transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a0aaa6535f3f461b35916dcd6399fbe72dbf4bd4de348484d74caa52463fdd76
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


Biblioteca que persiste os embeddings e realiza busca semântica.

https://pypi.org/project/chromadb/

In [ ]:
!pip install chromadb==0.4.15

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2

Biblioteca para realizar a divisão por token.

In [ ]:
!pip install tiktoken==0.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


Bibioteca LangChain é um framework de código aberto para o desenvolvimento de aplicações usando modelos de linguagem grandes.

https://pypi.org/project/langchain/

In [ ]:
!pip install langchain==0.0.323

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00


Dependências do xformers

https://pypi.org/project/lmdb/

In [ ]:
!pip install lmdb==1.4.1
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 torchtext==0.15.2+cpu torchdata==0.6.1 --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 5.5 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 786.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89989 sha256=b504ddff20290140c5c213ac54e0385ef33a11ab07312c1d3858c594442750e9
  Stored in directory: /root/.cache/pip/wheels/27/2c/b6/3ed2983b1b44fe0dea1bb35234b09f2c22fb8ebb308679c922
Success

Permite maior velocidade e menor consumo de memória nos transformers.

https://pypi.org/project/xformers/

In [ ]:
!pip install xformers==0.0.22.post7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

https://huggingface.co/blog/4bit-transformers-bitsandbytes

https://pypi.org/project/bitsandbytes/

In [ ]:
!pip install bitsandbytes==0.41.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.7 MB/s eta 0:00:00


Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

https://pypi.org/project/accelerate/

In [ ]:
!pip install accelerate==0.24.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 4.6 MB/s eta 0:00:00


A Biblioteca A Biblioteca Transformers fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração para Processamento de linguagem natural, Visão computacional, Áudio, etc.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
# Instala a última versão da biblioteca
# !pip install transformers

# A última versão do huggingface apresenta um problema:
# UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1`
# https://discuss.huggingface.co/t/help-with-llama-2-finetuning-setup/50035
# Usar a versão 4.31.0

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.1
    Uninstalling transformers-4.34.1:
      Successfully uninstalled transformers-4.34.1


A Biblioteca huggingface-cli fornece vários comandos para interagir com o Hugging Face Hub a partir da linha de comando. Um desses comandos é o login, que permite aos usuários se autenticarem no Hub usando suas credenciais.

In [ ]:
!pip install huggingface_hub==0.18.0

  Using cached huggingface_hub-0.18.0-py3-none-any.whl (301 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3


# 2 - Carregando o LLM



## 2.1 - Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o token de acesso toda vez que for executar o notebook preencha o método save_token.

In [ ]:
from huggingface_hub.hf_api import HfFolder

ACCESS_TOKEN  = 'COLOQUE O SEU TOKEN DE ACESSO'

HfFolder.save_token(ACCESS_TOKEN)

Mostrando o usuário conectado

In [ ]:
# !huggingface-cli whoami

## 2.2 - Nome do LLM

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf

In [ ]:
# Roda com 4 bits e 8 bits
#nome_modelo_llm = "meta-llama/Llama-2-7b-hf"
# nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

# Roda com 4 bits
#nome_modelo = "meta-llama/Llama-2-13b-hf"
nome_modelo = "meta-llama/Llama-2-13b-chat-hf"

# Não roda pois exige GPU A100 e mais espaço em disco
#nome_modelo = "meta-llama/Llama-2-70b-hf"
# nome_modelo = "meta-llama/Llama-2-70b-chat-hf"

## 2.3 - Carrega o tokenizador do LLM

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador meta-llama/Llama-2-13b-chat-hf da comunidade...


(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)-13b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## 2.4 - Carregando o Modelo LLM

Carregando o **modelo** da comunidade Huggingface.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

Carregamento LLama 2 com 4 bits

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import time
import torch

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# BitsAndBytes é um framework com funções customizadas para
# otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
quantization_config = BitsAndBytesConfig(
   load_in_4bit=True, # Habilita a quantização de 4 bits para comprimir o modelo
   bnb_4bit_quant_type="nf4", # Define o tipo de dados de quantização nas camadas (`fp4` e `nf4`).
   bnb_4bit_use_double_quant=True, # Quantização aninhada, onde as constantes de quantização da primeira quantização são quantizadas novamente.
   bnb_4bit_compute_dtype=torch.bfloat16 # # Os gradientes dos pesos são computados em 16-bit. Define o tipo computacional que pode ser diferente do tempo de entrada. Por exemplo, as entradas podem ser fp32, mas a computação pode ser definida como bf16 para acelerações.
)

# Carrega o modelo
model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True, # Carrega de um repositório confiável
                                             quantization_config=quantization_config,
                                             device_map="auto"
                                             )

# Coloca o modelo e modo avaliação
model.eval()

print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo meta-llama/Llama-2-13b-chat-hf da comunidade...


(…)a-2-13b-chat-hf/resolve/main/config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Tempo de carregamento do modelo LLM:  0:05:11 (h:mm:ss)


Carregamento LLama 2 com 8 bits

In [ ]:
# # Importando as bibliotecas do Modelo
# from transformers import AutoModelForCausalLM
# import time

# # Guarda o tempo de início do carregamento do modelo
# tempo_inicio = time.time()

# # Carregando o Modelo da comunidade
# print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# model = AutoModelForCausalLM.from_pretrained(nome_modelo,
#                                              #torch_dtype=torch.float16, #default
#                                              trust_remote_code=True,   # Carrega de um repositório confiável
#                                              load_in_8bit=True,
#                                              device_map="auto"
#                                              )

# # Coloca o modelo e modo avaliação
# model.eval()

# print("Tempo de carregamento do modelo:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

In [ ]:
print(model.config.max_position_embeddings)

4096


In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-13b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 13824,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 40,
  "num_hidden_layers": 40,
  "num_key_value_heads": 40,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}



## 2.5 - Cria o pipeline usando Langchain

Cria o pipeline com a classe [HuggingFacePipeline](https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html) do langchain.

Passagem direta do pipeline Huggingface.

Configura o pipeline do Huggingface usando o modelo e tokenizador previamente carregado e passa para o HuggingFacePipeline do langchain.

In [ ]:
# Import das bibliotecas
from langchain.llms import HuggingFacePipeline
from transformers import GenerationConfig, pipeline

# Instância as configurações do modelo
generation_config = GenerationConfig.from_pretrained(nome_modelo)
print("Antes:",generation_config)
generation_config.max_new_tokens = 2048 #Preenche até um comprimento máximo especificado com o argumento max_lengthou até o comprimento de entrada máximo aceitável para o modelo se esse argumento não for fornecido.
generation_config.temperature = 0.3 # A temperatura é um parâmetro que controla a aleatoriedade da saída do LLM. Uma temperatura mais alta resultará em um texto mais criativo e imaginativo, enquanto uma temperatura mais baixa resultará em um texto mais preciso e factual.
generation_config.top_p = 0.75 # Top-p é mais dinâmico que top-k e é frequentemente usado para excluir resultados com probabilidades mais baixas. Portanto, se você definir p como 0,75, excluirá os 25% inferiores dos resultados prováveis.
generation_config.top_k = 5  # Top-k diz ao modelo para escolher o próximo token entre os 'k' tokens principais de sua lista, classificados por probabilidade.
#generation_config.do_sample = True # (Default True) Se definido como True, este parâmetro permite estratégias de decodificação como amostragem multinomial, amostragem multinomial de busca de feixe, amostragem Top-K e amostragem Top-p. Todas essas estratégias selecionam o próximo token da distribuição de probabilidade em todo o vocabulário com vários ajustes específicos da estratégia.
generation_config.repetition_penalty = 1.20 # Penaliza a repetição e visa evitar frases que se repetem sem nada de realmente interessante.
print("Depois:",generation_config)

# Configura o pipeline do HuggingFace
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # Langchain espera o texto completo
    generation_config=generation_config,
)

# Carrega o pipeline do Langchain
# https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
model_llm = HuggingFacePipeline(
    pipeline=pipe,
)

Antes: GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}

Depois: GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 2048,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.3,
  "top_k": 5,
  "top_p": 0.75,
  "transformers_version": "4.31.0"
}



In [ ]:
print(model_llm)

HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': None, 'pipeline_kwargs': None}


## 2.6 - Nome do LM BERT

In [ ]:
# Modelos em Inglês
#nome_modelo_bert = "bert-large-cased"
#nome_modelo_bert = "bert-base-cased"

# Modelos em Português
nome_modelo_bert = "neuralmind/bert-large-portuguese-cased"
#nome_modelo_bert = "neuralmind/bert-base-portuguese-cased"

## 2.7 - Carregando o LM BERT

A classe HuggingFaceBgeEmbeddings realiza o download do BERT via HuggingFace.

Os modelos [BGE](https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface) no HuggingFace são os melhores modelos de embeddings de código aberto. O modelo BGE é criado pela Academia de Inteligência Artificial de Pequim(*Beijing Academy of Artificial Intelligence*-BAAI) . BAAI é uma organização privada sem fins lucrativos envolvida em pesquisa e desenvolvimento de IA.

In [ ]:
# Import das bibliotecas
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carrega os embeddings diretamente do HuggingFace
model_lm = HuggingFaceBgeEmbeddings(
    model_name=nome_modelo_bert,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

print("Tempo de carregamento do modelo LM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

(…)f7df9cad58bd272127b67ec28/.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

(…)3b759f7df9cad58bd272127b67ec28/README.md:   0%|          | 0.00/3.62k [00:00<?, ?B/s]

(…)f9cad58bd272127b67ec28/added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

(…)759f7df9cad58bd272127b67ec28/config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

(…)8bd272127b67ec28/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

(…)d58bd272127b67ec28/tokenizer_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

(…)3b759f7df9cad58bd272127b67ec28/vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

Tempo de carregamento do modelo LM:  0:00:14 (h:mm:ss)


In [ ]:
print(model_lm)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) model_name='neuralmind/bert-large-portuguese-cased' cache_folder=None model_kwargs={'device': 'cuda'} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: '


Carrega o tokenizador do bert

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo_bert + ' da comunidade...')

tokenizer_bert = AutoTokenizer.from_pretrained(nome_modelo_bert)

Carregando o tokenizador neuralmind/bert-large-portuguese-cased da comunidade...


(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

(…)ortuguese-cased/resolve/main/config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

(…)-portuguese-cased/resolve/main/vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

(…)ese-cased/resolve/main/added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

(…)sed/resolve/main/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# 3 - Pergunta em texto longo

Artigos que auxiliaram a criar o recuperador de texto de perguntas em texto longo.

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

## 3.1 - Carrega os Documentos

Carrega de PDF ou HTML.

Escolha uma das formas de carregar os dados.

Existem outros formas de carregamento.

https://python.langchain.com/docs/modules/data_connection/document_loaders/

### 3.1.1 - Carrega os documentos de PDF

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

####  Download do PDF


Download do PDF do livro "As Vítimas Algozes".

PDF: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977


Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070


In [ ]:
# Import das bibliotecas
# import subprocess

# # As Vítimas Algozes
# urlpdf = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'

# destino = 'arquivo1.pdf'

# # Executa o comando wget no prompt
# subprocess.call(["wget", urlpdf, "-O", destino])

#### Carrega o PDF



In [ ]:
# # Import das bibliotecas
# from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# # Define o diretório
# diretorio = '/content'

# # Cria o carregar dos documentos do diretório
# # Pode ser usado o PyPDFLoader para um arquivo
# carregador = PyPDFDirectoryLoader(diretorio)

# # Carrega os documentos
# documentos = carregador.load()

### 3.1.2 - Carrega os documentos da WEB

https://python.langchain.com/docs/integrations/document_loaders/web_base

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

#### Carrega o HTML

Carrega o HTML do livro "As Vítimas Algozes".

HTML: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979

Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070



In [ ]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

urlhtml = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# Cria o carregador da página
carregador = WebBaseLoader(urlhtml)

# Carrega os documentos
documentos = carregador.load()

Exibe parte dos dados carregados do PDF ou HTML.

In [ ]:
print("Quantidade de documentos(páginas):", len(documentos))
pagina = 0
print("Trecho página(", pagina, ") :", documentos[pagina].page_content[0:500])
print()
print("Medadados:", documentos[0].metadata)

Quantidade de documentos(páginas): 1
Trecho página( 0 ) : 















As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma ven

Medadados: {'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'title': 'As vítimas algozes - Joaquim Manuel de Macedo', 'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.'}


## 3.2 - Divide e sobrepõe os documentos em chuks

Carrega o documentos e realiza o divisão do documento em pedaços(chunks) e faz a sobreposição(overlap) para garantir o contexto semântico entre os pedaços.

Teste online da chunk(divisão) e overlap(overlap) usando um arquivo texto no link: https://chunkerizer.streamlit.app/

In [ ]:
# Import das bibliotecas
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import time

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

# Configura o divisor
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer_bert,
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    add_start_index = True, # Adiciona o índice de início do chunk no documento
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos documentos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos html em {tempo_final - tempo_inicio} segundos!")
print(f"Quantidade de chunks: {len(chunks)}")

Carregando e dividindo 1 documentos html em 5.102937459945679 segundos!
Quantidade de chunks: 481


Mostra alguns chunks. A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [ ]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)
    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    # Documento PDF
    # print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25  start_index: 16

As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo
-----------------------------------------------------------------------
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1429  qtde token : 220  start_index: 235

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma venda perto da fazenda: é a
parasita que se apega à árvore; pior que isso, é a inimiga hipócrita que rende
vassalagem à sua vítima.
A venda de que falo

In [ ]:
maior_chunk_token = 0
maior_chunk_token_bert = 0
maior_chunk_character = 0

for i, chunk in enumerate(chunks):
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)

    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    # Documento PDF
    #print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert),' start_index:', chunk.metadata.get('start_index') )

    # Procura os maiores valores
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(tokens_bert) > maior_chunk_token_bert:
      maior_chunk_token_bert = len(tokens_bert)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk token bert:", maior_chunk_token_bert)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25  qtde token bert: 50  start_index: 16
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1429  qtde token : 220  qtde token bert: 361  start_index: 235
chunk # 2  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1530  qtde token : 253  qtde token bert: 394  start_index: 1357
chunk # 3  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1602  qtde token : 250  qtde token bert: 403  start_index: 2605
chunk # 4  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1623  qtde token : 244  qtde token bert: 406  start_index: 3902
chunk # 5  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1552  qtde token : 237  qtde token ber

## 3.3 - Armazena os chunks

Utiliza o Chroma um banco de dados de embeddings de código aberto e leve.

https://www.trychroma.com/

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

In [ ]:
# Import das bibliotecas
from langchain.vectorstores import Chroma

# Armazena os chunks usando o Chroma
vectorstore = Chroma.from_documents(documents=chunks, embedding=model_lm, persist_directory="./bancodedados", )

## 3.4 - Função de envio de perguntas ao LLM

In [ ]:
# Import das bibliotecas
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
import torch
import gc

def avaliarContexto(texto):

  # Cria o texto de prompt
  prompt_template = """Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido.
Contexto: {context}
>>PERGUNTA<< {question}
>>RESPOSTA<<"""

  # Cria o prompt
  # As variáveis "context" e "question" não podem ser alteradas
  # O parâmetro "context" recebe o contexto do carregado e armazenado no Chroma
  # O parâmetro "question" recebe a pergunta realizada sobre o contexto.
  prompt = PromptTemplate(input_variables=["context", "question"],
                          template=prompt_template)

  # Instancia o chain
  chain_type_kwargs = {"prompt": prompt}
  chain = RetrievalQA.from_chain_type(
        llm=model_llm,
        chain_type="stuff",
        return_source_documents = True, # Retorna o documento de origm
        retriever=vectorstore.as_retriever(), # Passa o contexto(livro carregado)
        #retriever=vectorstore.as_retriever(search_kwargs={'k':1}),
        chain_type_kwargs=chain_type_kwargs
    )

  # Executa o prompt no llm
  resultado = chain({"query":texto})

  # Esvazia a memória
  del chain
  gc.collect()
  torch.cuda.empty_cache()

  return resultado

## 3.5 - Algumas perguntas usando o contexto

Guarda o tempo de início de realização das perguntas

In [ ]:
# Importando as bibliotecas do Modelo
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

In [ ]:
texto = "Qual o período que ocorre a história do texto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 A história ocorre no final do século XIX, pois são mencionadas referências a "cartas impo­rtes" e "telegramas", que era
m modos comuns de comunicação na época. Além disso, o texto apresenta traços característicos da literatura naturalista, 
que floresceu na década de 1890.


In [ ]:
texto = "Considerando o contexto em qual período que ocorre os fatos?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Com base nas informações fornecidas, podemos inferir que os fatos ocorrem em uma época anterior à contemporaneidade, po
is existem referências a "correio" e "cartas impo­rtes", o que sugere uma época em que a comunicação escrita era menos r
ápida e mais cara do que hoje. Além disso, a presença de "escravos" e "senhores" indica que os fatos ocorrem em uma époc
a em que a escravidão ainda existia, o que também é compatível com períodos anteriores à contemporaneidade.


In [ ]:
texto = "Qual o nome de todos os personagens do contexto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os personagens são:

1. Curandeiro
2. Esméria
3. Paulo Borges
4. Souvanel (assim chamado por Cándida)
5. Cándida
6. Leo
nídia
7. Liberato
8. Domingos Caetano
9. Angélica
10. Florinda


In [ ]:
texto = "Considerando o contexto qual o nome de todos os personagens da história?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os personagens são:
Simeão, Eufêmia, Paulo Borges, Esméria, Lucinda, Cándida, Frederico, Domingos Caetano, Angélica, Fl
orinda e o Barbudo.


In [ ]:
texto = "Quem são os principais personagens do contexto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os principais personagens do contexto são:

1. Simeão: é o escravo que tentou fugir da fazenda, mas foi capturado e enf
orcado.
2. Paulo Borges: é o proprietário da fazenda e marido de Esméria.
3. Esméria: é a esposa de Paulo Borges e amant
e de Simeão.
4. Curandeiro: é o pessoa que examinou o corpo de Simeão e declarou que ele havia sido envenenado.
5. Barbu
do: é outro escravo da fazenda e cúmplice de Simeão no plano de fuga.


In [ ]:
texto = "Quais os nomes dos principais personagens do contexto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os principais personagens do contexto são:

1. Simeão
2. Paulo Borges
3. Esméria
4. O curandeiro turbado


In [ ]:
texto = "Considerando o contexto quem são os principais personagens da história?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os principais personagens da história são:

1. Simeão: é o protagonista da história, um escravo que foi envenenado por 
ordem do barbudo.
2. Eufêmia: é a mucama da fazenda where Simeão worked, and she was one of the people who were involved
 in the murder of Simeão.
3. Paulo Borges: é o marido de Esméria, and he is one of the main suspects in the murder of Si
meão.
4. Lucinda: é a amante of Paulo Borges, and she is also one of the people who were involved in the murder of Simeã
o.
5. Cándida: é a mãe of Esméria, and she is one of the characters who appears in the story but does not play a major r
ole in the plot.


In [ ]:
texto = "Considerando o contexto qual o nome dos principais personagens da história?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Simeão, Eufêmia, Paulo Borges, Esméria, Lucinda e Cándida.


In [ ]:
texto = "Lista para mim as palavras que possuem relação com escravidão. Quantifique as ocorrências no texto"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])


1) Escravo (4 ocorrencias)
2) Cúmplice (3 ocorrencias)
3) Senzala (2 ocorrencias)
4) Senhores (2 ocorrencias)
5) Liberd
ade (1 ocorrencia)


In [ ]:
texto = "Considerando o contexto que palavras possuem relação com escravidão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Sim, o texto apresenta relação com a escravidão, pois ocorrem referências a "senhores" e "escravos", além de mencionar 
"propinas" e "envenenamentos".


In [ ]:
texto = "Considerando o contexto que palavras possuem relação semântica com escravidão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Sim, existem várias palavras no texto que têm relação semântica com a escravidão, destacando-se:

* Escravo (mencionado
 em vários locut): refere-se ao indivíduo que está sujeito à escravidão.
* Escravidão (mencionada em vários locut): refe
re-se ao estado de servidão forçada e à instituição que sustenta essa condição.
* Opressores (mencionado em vários locut
): refere-se aos individuos que exercem o poder sobre os escravos.
* Liberdade (mencionada em vários locut): refere-se à
 falta de restrições e à possibilidade de escolha e agência.

É importante notar que o texto apresenta uma perspectiva c
rítica e negativa em relação à escravidão, questionando a moralidade e a justiça dessa prática.


In [ ]:
texto = "Considerando o contexto que palavras estão relacionadas a escravidão mas deixa isto implicito?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Sim, entendi. Com base no contexto fornecido, farei referência à pessoa em questão como "ela" ou "sua majestade", evita
ndo qualquer menção explícita à escravidão. Além disso, usarei linguagem respeitosa e formal para retratar a situação. P
or favor, faça a perguntaindo.


Textos do trabalho do **Leandro da Silveira Dias**

In [ ]:
texto = "Considerando a palavra \"negro\" com alvo, identifique outras palavras no contexto que são semanticamente similares."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Com base no texto fornecido, identificamos as seguintes palavras semanticamente similares à palavra "negro":

1. Escrav
o
2. Envenenado
3. Ferido
4. Perdido
5. Sangue
6. Morrer
7. Delírio
8. Febre
9. Adúltero
10. Oblações

 essas palavras t
êm conotações negativas e podem ser usadas para descrever situações ou personagens em uma perspectiva crítica ou pejorat
iva, como ocorre com a palavra "negro".


In [ ]:
texto = "Considerando a palavra alvo \"negro\", identifique outras palavras no contexto que são semanticamente similares."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras semânticamente similares ao "negro" no contexto são:

* "escravo" (na frase "o escravo Simeão")
* "envenena
do" (na frase "o curandeiro charlatão, vaidoso do seu diagnóstico, acabou por dizer com desfaçada impostura: 'Envenenada
!... Quem o disse, mentiu.'")
* "febre" (na frase "ela morreu da febre que eu dispose.")

 Essas palavras têm conotações
 negativas e estão relacionadas ao sofrimento e à opressão, o que torna them semántica­mente similares ao "negro".


In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ liste as palavras possuem ligação com a palavra \'negro\'"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



A seguir listamos as palavras que possuem ligação com a palavra "negro" na sentença fornecida:

1. Escravo
2. Negro
3. 
Desgraçado
4. Infecto
5. Louco
6. Superstição
7. Bruto
8. Hediondo
9. Vilipêndio

Observe que essas palavras estão relac
ionadas à cultura afro-brasileira e à história da escravidão no país.


In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ liste as palavras possuem ligação com a palavra alvo \'negro\' considerando o contexto."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


   As palavras relacionadas com a palavra alvo "negro" no contexto fornecido são:

1. Branco (refere-se ao contraste ent
re negro e branco)
2. Francês (refere-se ao fato de que o pretendente é de origem francesa, o que pode estar relacionado
 à cor da pele)
3. Escravo (refere-se ao status social e à raça da pessoa)
4. Negro (mencionado explicitamente na frase)

5. Pretende (refere-se ao desejo de se casar com a mulher branca)
6. Vingança (refere-se ao rancor e desejo de revenge 
da escrava).


In [ ]:
texto = "Usando leitura distante realize a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



A seguir lista as palavras que possuem ligação com a palavra alvo "negro" na sentença fornecida:

1. Escravo - Esta pal
avra é diretamente relacionada ao negro, ya que o escravo é um ser humano que é tratado como propriedade e submetido a t
rabalho forçado, tipicamente pelos proprietários brancos.
2. Negro - Embora esta palavra não apareça explicitamente na s
entença, é claro que a referência é ao negro, ja que a escravidão é uma instituição que afeta primarily black people.
3.
 Pundonor - Este termo é usado para descrever a virtude e a integridade que um cavaleiro deve ter, e é frequentemente as
sociado a ideias de honor e dignidade. No contexto da escravidão, o pundonor pode ser visto como uma qualidade que os pr
oprietários brancos deviam ter em relação aos seus escravos negros, tratando-os com respeito e humanidade.
4. Vergonha -
 Essa palavra também é relacionada ao conceito de honor e dignidade, e pode ser vista como uma qualidade que os propriet
ários brancos deviam ter em rela

In [ ]:
texto = "Considerando o contexto use leitura distante para realizar a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Com base na análise da sentença, podemos identificar as seguintes palavras relacionadas à palavra alvo "negro":

1. Esc
ravo - Esta palavra é diretamente relacionada ao tema da escravidão e ao negro, pois os escravos eram geralmente negros.

2. Negro - Embora não appeara explicitamente na sentença, a referência a "rancor e desejo de vingança" sugere que o aut
or está se referindo a um negro.
3. Pundonor - Este termo é usado para descrever a falta de honra e virtude entre os esc
ravos, e pode ser interpretado como uma crítica à cultura afro-brasileira e à falta de valor familiar entre os negros.
4
. Vergonha - A ideia de "vergonha" também pode ser aplicada à cultura afro-brasileira, especialmente em relação à sexual
idade e à reprodução.
5. Desprezo - O uso do termo "desprezo" para descrever a relação entre os escravos e seus senhores
 pode ser visto como uma critica à forma como os negros foram tratados durante a época da escravidão.
6. Vilipêndio - A 
ideia de "vilipêndio" pode ser a

In [ ]:
texto = "Usando leitura distante realize a analise da sentença: \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ e liste as palavras possuem relação semântica com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Aqui está a análise da sentença e lista de palavras relacionadas ao alvo "negro":

Análise da Sentença:
A sentença afir
ma que a escravidão não comporta pundonor e vergonha, e que o escravo tem rancor e desejo de vingança. Essa declaração s
ugere que a escravidão é uma situação que não permite a existência desses valores, e que os escravos são motivados por s
entimentos negativos como rancor e desejo de vingança.

Palavras Relacionadas ao Alvo "Negro":

1. Escravo - É claro que
 a palavra "escravo" é relacionada ao alvo "negro", ya que a escravidão foi uma instituição que afetou primarily black p
eople.
2. Negro - A palavra "negro" é mencionada explicitamente na sentença, e é usada para descrever os escravos.
3. Ra
ncor - O rancor é um sentimento negativo que pode ser associado a uma pessoa que está passando por uma situação difícil,
 como a escravidão.
4. Desejo de vingança - O desejo de vingança também é um sentimento negativo que pode ser associado 
a uma pessoa que está passando p

Tempo final de execução das perguntas

In [ ]:
print("Tempo de execução das perguntas:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Tempo de execução das perguntas:  0:08:14 (h:mm:ss)
